In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import hitnet_trafo

In [2]:
data = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols')

In [3]:
hits_input = tf.keras.Input(shape=(5,))
params_input = tf.keras.Input(shape=(len(data.labels),))
t = hitnet_trafo(labels=data.labels)

h = t(hits_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(1024, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

hit_net = tf.keras.Model(inputs=[hits_input, params_input], outputs=outputs)
hit_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo (hitnet_trafo)     (None, 15)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           512         hitnet_trafo[0][0]               
______________________________________________________________________________________________

In [4]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hit_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [5]:
train_id = 'HitNet_1024_L3' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [6]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [7]:
train_data, test_data = data.get_hitnet_data(train_batch_size=1024,
                                             test_batch_size=1024,
                                            )

In [ ]:
hist = hit_net.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                  initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
75802/75802 - 707s - loss: 0.3560 - val_loss: 0.0000e+00
Epoch 2/1000
75802/75802 - 346s - loss: 0.3318 - val_loss: 0.3221
Epoch 3/1000
75802/75802 - 341s - loss: 0.3176 - val_loss: 0.3045
Epoch 4/1000
75802/75802 - 338s - loss: 0.3025 - val_loss: 0.2909
Epoch 5/1000
75802/75802 - 340s - loss: 0.2907 - val_loss: 0.2799
Epoch 6/1000
75802/75802 - 341s - loss: 0.2820 - val_loss: 0.2734
Epoch 7/1000
75802/75802 - 341s - loss: 0.2754 - val_loss: 0.2664
Epoch 8/1000
75802/75802 - 342s - loss: 0.2702 - val_loss: 0.2641
Epoch 9/1000
75802/75802 - 340s - loss: 0.2662 - val_loss: 0.2608
Epoch 10/1000
75802/75802 - 362s - loss: 0.2631 - val_loss: 0.2568
Epoch 11/1000
75802/75802 - 340s - loss: 0.2606 - val_loss: 0.2546
Epoch 12/1000
75802/75802 - 342s - loss: 0.2586 - val_loss: 0.2537
Epoch 13/1000
75802/75802 - 340s - loss: 0.2569 - val_loss: 0.2514
Epoch 14/1000
75802/75802 - 339s - loss: 0.2555 - val_loss: 0.2518
Epoch 15/1000
75802/75802 - 341s - loss: 0.2543 - val_loss: 0.2506


In [ ]:
# L7 0.28
# L3 0.24